Allen Institute resources for API access:
- http://help.brain-map.org/display/api/Allen+Brain+Atlas+API
- http://help.brain-map.org/display/api/Downloading+3-D+Expression+Grid+Data
- http://help.brain-map.org/display/api/Example+Queries+for+Experiment+Metadata
- http://help.brain-map.org/pages/viewpage.action?pageId=5308449

XML parsing:
- https://www.datacamp.com/community/tutorials/python-xml-elementtree
- https://www.kite.com/python/answers/how-to-download-a-csv-file-from-a-url-in-python


In [2]:
import requests
import xml.etree.ElementTree as ET

In [29]:
# url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::SectionDataSet,rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse'],treatments[name$eq'ISH'],genes,plane_of_section,rma::options,[tabular$eq'plane_of_sections.name+as+plane','genes.acronym+as+gene','data_sets.id+as+section_data_set_id'],[order$eq'plane_of_sections.name,genes.acronym,data_sets.id']&num_rows=50&start_row=0"
url = "http://api.brain-map.org/api/v2/data/query.xml?criteria=model::SectionDataSet,rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse'],treatments[name$eq'ISH'],rma::include,genes,specimen(donor(age)),plane_of_section&num_rows=200&start_row=0"

In [ ]:
#This XML query gets the 4345 experiments in the coronal data set
"http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH]"

#This one gets the 21733 experiments in the sagittal data set
"http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqsagittal],products[abbreviation$eqMouse],treatments[name$eqISH]"

In [ ]:
# abi_brainmap_base = "http://api.brain-map.org/api/v2/data"
# abi_brainmap_model = "/SectionDataSet"
# abi_brainmap_queryxml = "/query.xml?"
# abi_brainmap_criteria = {}

In [13]:
root = ET.fromstring(r.content)

In [10]:
root.tag

'Response'

In [14]:
root.attrib

{'success': 'true', 'start_row': '0', 'num_rows': '1', 'total_rows': '4345'}

In [15]:
abi_brainmap_query_init = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH]&start_row=0&num_rows=1"

r = requests.get(abi_brainmap_query_init)

root = ET.fromstring(r.content)
nfiles = int(root.attrib['total_rows'])

In [18]:
url = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&start_row=4340&num_rows=10"


# %20rma::options,%20[tabular$eq%27plane_of_sections.name+as+plane%27,%27genes.acronym+as+gene%27,%27data_sets.id+as+section_data_set_id%27],%20[order$eq%27plane_of_sections.name,genes.acronym,data_sets.id%27]%20&num_rows=50&start_row=0"

In [3]:
import pandas as pd

In [19]:
pd.read_csv(url)

,gene,experimentid,plane
0,Cyp26b1,72081548,coronal
1,Kcng4,72081560,coronal
2,Glra1,72108824,coronal
3,Ace,72109489,coronal
4,Rnf13,72109423,coronal


In [27]:
abi_query_base = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&"+\
"tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&"

# start_row=4340&num_rows=10"

num_rows = 100
start_row = 0
end_row = start_row + num_rows - 1
flag = 1

In [28]:
abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)

In [16]:
nfiles = 5

4345

In [17]:
while flag == 1:
    abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)
    
    

True

In [41]:
abi_query_base = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&"+\
"tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&"

num_rows = 'all'
abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)
temp = pd.read_csv(abi_query)

In [42]:
temp

,gene,experimentid,plane
0,Nr2f2,112646890,coronal
1,Tbc1d8,71836747,coronal
2,Kcng3,71717451,coronal
3,Phox2b,2480,coronal
4,Thbs2,73929578,coronal
...,...,...,...
4340,Cyp26b1,72081548,coronal
4341,Kcng4,72081560,coronal
4342,Glra1,72108824,coronal
4343,Ace,72109489,coronal


In [45]:
temp_id = temp.loc[0, 'experimentid']

In [51]:
url = 'http://api.brain-map.org/grid_data/download/{}'.format(temp_id)
url

'http://api.brain-map.org/grid_data/download/112646890'

In [52]:
r = requests.get(url)


In [54]:
outfile = 'Data/Expression/temp.zip'
with open(outfile, 'wb') as file:
    file.write(r.content)